In [5]:
import pyodbc
import pandas as pd
import numpy as np

import plotly.express as px

from datetime import datetime

In [6]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.float_format = '{:.2f}'.format

# Load data

In [7]:
def get_df(query):
    server = 'OmerSteven\SQLEXPRESS'
    database = 'AdventureWorksDW2019'
    conn = pyodbc.connect('Driver={SQL Server};'
                      f'Server={server};'
                      f'Database={database};'
                      'Trusted_Connection=yes;')
    df = pd.read_sql(query,conn)
    
    # Close the connection
    conn.close()
    return df

In [8]:
query = """
            SELECT * 
            FROM AdventureWorksDW2019.dbo.DimCustomer
            """

# Output of query to df
analysis_tabel = get_df(query)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


# Analysis

In this analysis we will have a look at the customer data of the AdventureWorks dataset. We are particularly interested in the yearly income distribution and how this relates to gender and age.

## Descriptive statistics

In [ ]:
descriptives_table = analysis_tabel\
    .groupby(['Gender'])\
    .agg({
        'CustomerKey' : 'count',
        'YearlyIncome' : ['mean', lambda x: x.mode(), 'max', 'min']
    })\
    .reset_index()

# list comprehension for columns
descriptives_table.columns = ['_'.join(col).replace('<lambda_0>', 'mode') for col in descriptives_table.columns]

descriptives_table

In [ ]:
# create table for value counts per gender
pie_chart_df = analysis_tabel.YearlyIncome\
                .value_counts()\
                .reset_index()\
                .rename(columns = {'index' : "yearly income",
                                   'YearlyIncome' : 'count'
                                  })

# pie chart
fig = px.pie(pie_chart_df, values='count', names='yearly income',
             title='Yearly income distribution', color_discrete_sequence=px.colors.sequential.RdBu
            )

fig.show()

In [ ]:
Distriubtion_yearly_income_gender = analysis_tabel\
    .groupby(['YearlyIncome','Gender'])\
    .agg({
        'CustomerKey' : 'count'}
    )\
    .unstack('Gender')

Distriubtion_yearly_income_gender

## Yearly income per customer gender and age

In [ ]:
scatterplot_df = analysis_tabel\
    [['FirstName', 'BirthDate', 'Gender', 'YearlyIncome']]\
    .assign(
        today = lambda df: pd.to_datetime(datetime.today(), format='%Y/%m/%d'),
    )

# Convert the date columns to datetime
scatterplot_df['BirthDate'] = pd.to_datetime(scatterplot_df['BirthDate'])

# Define a lambda function to calculate the number of years in the date delta
Age = lambda df: divmod((df['today'] - df['BirthDate']).total_seconds(), 31536000)[0]

# Apply the lambda function to create a new column
scatterplot_df['Age'] = scatterplot_df.apply(Age, axis=1)

In [ ]:
fig = px.scatter(
        scatterplot_df[['FirstName', 'BirthDate', 'Gender', 'Age', 'YearlyIncome']], 
        x='Age', 
        y='YearlyIncome',
        hover_name='FirstName', 
        color = 'Gender',
        color_continuous_scale='redor'
    )

fig.show()

# Conclusion

Based on the dataset we can conclude the following:
•	There are slightly more female customers than male customers (52% vs 48%)<br>
•	The max yearly income salary is equal for both genders<br>
•	Although the max yearly income occurs more often among the male customers, the average income is more or less equal (even
    slightly higher for females).<br> 
    &nbsp;&nbsp;The mode yearly income for both genders is 60k.<br>
•	Based on the scatterplot there does not seem to be a significant correlation between yearly income and age.
